In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import warnings
warnings.filterwarnings("ignore")
import geopandas as gpd
import seaborn as sns
import copy

In [25]:
#please input the market data location in your computer
df = pd.read_excel("/Users/zhangming/energy-market-and-battery-optimization-group-25/data/market_data.xlsx")

In [26]:
## preprocessing to get date into the correct data type
df['Time (UTC+10)'] = pd.to_datetime(df['Time (UTC+10)'])
df = df[["Time (UTC+10)","Regions VIC Trading Price ($/MWh)"]]



In [27]:
## change time to only hours to see distribution based on groupby

##a = pd.to_datetime(a)
##a = df['Time (UTC+10)'].dt.strftime('%Y-%m-%d')
## change time to only hours to see distribution based on groupby
##b = (df['Time (UTC+10)'] - a).dt.total_seconds()//3600
##df['Time (UTC+10)'] = b
##df2 = df.groupby('Time (UTC+10)')['Regions VIC Trading Price ($/MWh)'].mean().reset_index()
##df2


In [28]:
## spliting dataset into 3 categories 
df_train = df.loc[(df["Time (UTC+10)"] < "2021-01-01 00:00:00" )]
df_cross = df.loc[(df["Time (UTC+10)"] >= "2021-01-01 00:00:00" )& (df["Time (UTC+10)"] < "2021-07-01 00:00:00" )]
df_test = df.loc[(df["Time (UTC+10)"] >= "2021-07-01 00:00:00" )]


In [29]:
## convert into numpy array for the algorithm input
df_train = df_train["Regions VIC Trading Price ($/MWh)"].to_numpy()
df_cross = df_cross["Regions VIC Trading Price ($/MWh)"].to_numpy()
df_test = df_test["Regions VIC Trading Price ($/MWh)"].to_numpy()
df_total = df["Regions VIC Trading Price ($/MWh)"].to_numpy()

In [30]:
## helper functions 
## df is the price list, index is the index of df, n is the length of the subsection of the df
## this solution works better if there are  multiple local min or max within the time frame
def compute_optimal_buyout(df, index, n):
    ## buyout refers to the dict which will be modified in the process as an output
    df_segment = df[index: index+n]
    buyout = {}
    ## contrust dictionary
    for i in range(0,len(df_segment)):
        buyout[i] = [df_segment[i],0] 

    ## sort dictionary
    temp = copy.deepcopy(buyout)
    sorted_buyout = {}
    sorted_keys = sorted(buyout, key=buyout.get)
    for i in sorted_keys:
        sorted_buyout[i] = temp[i]
    constraint = [644.4] * n
    ## now find profitable trade deals, and lastly apply contraint
    for i in sorted_buyout:
        remaining = 150
        if sorted_buyout[i][0] * 1.33 > max(df_segment):
            break
        ## for each min price, if you see a profitable price, do something
        for j in range(i, n):
            if remaining <= 0:
                break
            if (sorted_buyout[i][0] * 1.33 < sorted_buyout[j][0] and sorted_buyout[i][0] +20 < sorted_buyout[j][0] and sorted_buyout[i][0] >= 0 and sorted_buyout[i][1] <= 0) or \
                (sorted_buyout[i][0] < 0 and sorted_buyout[i][0] / 1.5 < sorted_buyout[j][0] and sorted_buyout[i][1] <= 0):
                index = j
                temp = j
                ## check if near solution has a immediate better solution from the next 2 time, and repeat if there are
                while(index< n-2):
                    
                    if max(sorted_buyout[index+1][0],sorted_buyout[index+2][0]) > sorted_buyout[index][0]:
                        if  max(sorted_buyout[index+1][0],sorted_buyout[index+2][0]) == sorted_buyout[index+1][0]:
                            temp = index + 1
                        elif  max(sorted_buyout[index+1][0],sorted_buyout[index+2][0]) == sorted_buyout[index+2][0]:
                            temp = index + 2
                        index = index + 2
                    else:
                        break
                ## index is time where we sell at the optimal price and now commence buyout and record it in buyout dict
                index = temp

                
                remaining_dispatch = 135 - sorted_buyout[index][1]
                
                ## remain is the max amount you can buy from the buy point
                ## remain dispatch is the max amount you can sell from the sell point
                ## hardcap is the max amount you can buy due to the limit of capacity
                ## the final amount of electricity you buy will be the most contrained value(minimum) out of all 3

                hardcap = min(150, remaining_dispatch/0.81,remaining)
                ## contraint list tracks all of the capacity during the timeframe
                ## if your deals makes any element fall below 0(the capacity will overcap), the deal need to be restricted. 
                for k in range(i,index + 1):
                    if constraint[k] <= 0:
                        hardcap = 0
                    elif (constraint[k] - min(150, remaining_dispatch/0.81,remaining) < 0) and constraint[k] >0:
                        hardcap = min(hardcap, constraint[k])
                
                for k in range(i,index ):
                    constraint[k] = constraint[k] - hardcap
                        
                remaining_dispatch = min(hardcap * 0.81,remaining * 0.81)
                    
                ## using the amount that you can buy, fill in the rest of the buyout using assumption from the spec
                if remaining_dispatch >= 150 * 0.81:
                    buyout[index][1] += 150 * 0.81

                else:
                    buyout[index][1] = min(135,buyout[index][1] + remaining_dispatch)
                
                buyout[i][1] = sorted_buyout[i][1] - (buyout[index][1] - sorted_buyout[index][1]) / 0.81

                

                remaining =  remaining - (-buyout[i][1] + sorted_buyout[i][1])


                sorted_buyout[i][1] = buyout[i][1]
                sorted_buyout[index][1] = buyout[index][1]
                
    
            

    last_buyout_index = 0
    for i in buyout:
        if buyout[i][1] > 0:
            last_buyout_index = i
            
    return [last_buyout_index,buyout]

In [31]:
## this version works better where there are centered local min and max in the time frame 
def compute_optimal_buyout_v2(df, index, n):
    ## buyout refers to the dict which will be modified in the process as an output
    df_segment = df[index: index+n]
    buyout = {}
    ## contrust dictionary
    for i in range(0,len(df_segment)):
        buyout[i] = [df_segment[i],0] 

    ## sort dictionary
    temp = copy.deepcopy(buyout)
    sorted_buyout = {}
    sorted_buyout_max = {}
    sorted_keys = sorted(buyout, key=buyout.get)
    sorted_keys_rev = sorted(buyout, key=buyout.get,reverse = True)
    for i in sorted_keys:
        sorted_buyout[i] = temp[i]
    for i in sorted_keys_rev:
        sorted_buyout_max[i] = temp[i]
    constraint = [644.4] * n
    ## now find profitable trade deals, and lastly apply contraint
    for i in sorted_buyout:
        remaining = 150
        if sorted_buyout[i][0] * 1.33 > max(df_segment):
            break
        for j in sorted_buyout_max:
            if remaining <= 0:
                break
            if ((sorted_buyout[i][0] * 1.33 < sorted_buyout[j][0] and sorted_buyout[i][0] >= 0 and sorted_buyout[i][1] <= 0) or \
                (sorted_buyout[i][0] < 0 and sorted_buyout[i][0] / 1.5 < sorted_buyout[j][0] and sorted_buyout[i][1] <= 0)) and j > i:
                index = j
                
                remaining_dispatch = 135 - sorted_buyout[index][1]
                
                ## apply contraint 
                hardcap = min(150, remaining_dispatch/0.81,remaining)
                
                for k in range(i,index + 1):
                    if constraint[k] <= 0:
                        hardcap = 0
                    elif (constraint[k] - min(150, remaining_dispatch/0.81,remaining) < 0) and constraint[k] >0:
                        hardcap = min(hardcap, constraint[k])
                
                for k in range(i,index ):
                    constraint[k] = constraint[k] - hardcap
                        
                remaining_dispatch = min(hardcap * 0.81,remaining * 0.81)
                    
                    
                if remaining_dispatch >= 150 * 0.81:
                    buyout[index][1] += 150 * 0.81

                else:
                    buyout[index][1] = min(135,buyout[index][1] + remaining_dispatch)
                
                ##buyout[i][1] = sorted_buyout[i][1] + (buyout[index][1] - sorted_buyout[index][1]) / 0.81
                buyout[i][1] = sorted_buyout[i][1] - (buyout[index][1] - sorted_buyout[index][1]) / 0.81

                
                ##remaining =  remaining - (buyout[i][1] - sorted_buyout[i][1])
                remaining =  remaining - (-buyout[i][1] + sorted_buyout[i][1])

                sorted_buyout[i][1] = buyout[i][1]
                sorted_buyout[index][1] = buyout[index][1]

    last_buyout_index = 0
    for i in buyout:
        if buyout[i][1] > 0:
            last_buyout_index = i
    return [last_buyout_index,buyout]

In [32]:
## using combination of all algorithm to find the best choice in the context
## add other model you think will benefit the algorithm in this part!
def emsemble_optimal_buyout(df, index, n):
    model1 = compute_optimal_buyout(df, index, n)
    model2 = compute_optimal_buyout_v2(df, index, n)

    dict1 = model1[1]
    dict2 = model2[1]
    if compute_profit(dict1) > compute_profit(dict2):
        return model1
    else:
        return model2

In [33]:
## the main function that compute the total profit, capacity and power usage
def main_function(df):
    total_profit = 0
    n = 0
    capacity = 0
    capacityy = []
    power = []
    while (n < len(df)):
        parameter = 48 - n % 24
        output = emsemble_optimal_buyout(df, n, min(parameter, len(df)-n))
        ## recompute parameter if the there are price brought at the last lookout
        ## add additional time data since there might be better buyout price
        if output[0] == parameter - 1:
            index = n+parameter-1
            if index + 1 < len(df):
                while df[index+1] > df[index] and index+1 < len(df):
                    index += 1
                output = emsemble_optimal_buyout(df, n, index - n + 1)
        n += output[0] +1
        total_profit += compute_profit(output[1])

        ## compute capacity and power during the timeframe 
        for i in output[1]:
            if output[1][i][1] < 0 and i < output[0] + 1:
                capacity += -output[1][i][1] * 0.9
                power.append(output[1][i][1] * 2)
                capacityy.append(capacity)
            elif output[1][i][1] >= 0 and i <output[0] + 1:
                capacity = capacity - output[1][i][1] / 0.9
                power.append(output[1][i][1] / 0.9 * 2)
                capacityy.append(capacity)

              
              


    return [capacityy, power,total_profit]
    

In [34]:
## buyout will be a dict that has value [price, amount]
def compute_profit(buyout):
    profit = 0
    for buy in buyout:
        if buyout[buy][1] < 0:
            profit += buyout[buy][0] * buyout[buy][1] /0.991
        else:
            profit += buyout[buy][0] * buyout[buy][1] * 0.991
    return profit

In [35]:
%%time
c = main_function(df_total)
c[2]

CPU times: user 7.57 s, sys: 11.3 ms, total: 7.58 s
Wall time: 7.6 s


126047962.70496097

In [36]:
## make it so that c[1] represent the opening capacity rather than closing capacity
c[0].insert(0,0)
del c[0][-1]

In [37]:
submission = pd.DataFrame()
submission["datetime"] = df["Time (UTC+10)"]
submission["power"] = c[1]
submission["capacity"] = c[0]

In [38]:
## creating the submission csv file
submission.to_csv("final_submission.csv", index = False)